In [ ]:
# 
import numpy as np
from numpy import arange
from matplotlib import pyplot
import pandas as pd
from pandas import  set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

#
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor

#
from mlxtend.regressor import StackingRegressor
from mlxtend.regressor import StackingCVRegressor

from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error

In [ ]:
#
dataset = pd.read_csv("paramaters.csv", index_col=0)
dataset.head()

In [ ]:
#
Y = dataset.Score
X = dataset.drop('Score', axis = 1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.3, random_state=7)

In [ ]:
# 
scoring1 = 'neg_mean_squared_error'
scoring2 = 'r2'
scoring1 = 'mean_absolute_error'

In [ ]:
#
lr = LinearRegression()
knn = KNeighborsRegressor(n_neighbors = 3)
dt = DecisionTreeRegressor(max_depth= 19, min_samples_leaf= 11, min_samples_split= 10)
rf = RandomForestRegressor(n_estimators = 700)

In [ ]:
#
models = [dt, knn, rf]
sclf = StackingCVRegressor(regressors=models, meta_regressor=lr)

In [ ]:
# 
print("stacking model")
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
sclf.fit(rescaledX, Y_train)
score = cross_val_score(sclf, rescaledX, Y_train, cv=5, scoring=scoring2)
print(score.mean(), "+/-", score.std())

In [ ]:
# Performance of the model on the Stacking Test Set Training Set
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
sclf = StackingCVRegressor(regressors=models, meta_regressor=lr)
sclf.fit(rescaledX, Y_train)

# Evaluate the training set
rescaledX_train = scaler.transform(X_train)
pre_train = sclf.predict(rescaledX_train)
print('Training_data，MSE：%s' % (mean_squared_error(Y_train, pre_train)))
print('Training_data，MAE：%s' % (mean_absolute_error(Y_train, pre_train)))
print('Training_data，R2：%s' %  (r2_score(Y_train, pre_train)))

# Evaluation test set
rescaledX_test = scaler.transform(X_test)
pre_test = sclf.predict(rescaledX_test)
print('Testing_data，MSE：%s' % (mean_squared_error(Y_test, pre_test)))
print('Testing_data，MAE：%s' % (mean_absolute_error(Y_test, pre_test)))
print('Testing_data，R2：%s' % (r2_score(Y_test, pre_test)))